Adjustable Parameters

In [ ]:
#numerical tolerance
tol = 1e-4

#number of restarts
res = 5

#name of the input txt file:
txt = "input.txt"

#name of the outout txt file:
output = "BFB.txt"

Code

In [1]:
# imports
import math
import numpy as np
from math import sin as s
from math import cos as c
from math import pi as pi

from numpy import exp
from numpy.random import default_rng

from scipy.optimize import minimize

from joblib import Parallel, delayed

from numba import njit

Defines the potential, in 4 different gauges: 

In [3]:
@njit(cache = True)
def v(x,l):
    t,a,b,y,d,e,n = x[0], x[1], x[2], x[3], x[4], x[5], x[6]
    
    phi1 = c(t)**2 + (s(t)*c(a))**2
    phi2 = (s(t)*s(a))**2*(c(b)**2 + (s(b)*c(y))**2)
    phi3 = (s(t)*s(a)*s(b)*s(y))**2
    phi12_R = 1/2*s(t)**2*s(2*a)*c(b)
    phi12_I = 1/2*s(t)**2*s(2*a)*s(b)*c(y)
    phi13_R = 1/2*(s(b)*s(y)*(s(2*t)*s(a)*c(d) + s(t)**2*s(2*a)*s(d)*s(e)*c(n)))
    phi13_I = 1/2*(s(b)*s(y)*s(d)*(s(2*t)*s(a)*c(e) + s(t)**2*s(2*a)*s(e)*s(n)))
    phi23_R = 1/2*(s(t)*s(a))**2*s(d)*s(e)*(s(2*b)*s(y)*c(n) + s(b)**2*s(2*y)*s(n))
    phi23_I = 1/2*(s(t)*s(a))**2*s(d)*s(e)*(s(2*b)*s(y)*s(n) - s(b)**2*s(2*y)*c(n))
    
    
    g1 = 1/2*l[0]*phi1**2 + 1/2*l[1]*phi2**2 + 1/2*l[2]*phi3**2 + l[3]*phi1*phi2 + l[4]*phi1*phi3 + l[5]*phi2*phi3
    g2 = l[6]*(phi12_R**2 + phi12_I**2) + l[7]*(phi13_R**2 + phi13_I**2) + l[8]*(phi23_R**2 + phi23_I**2)
    g3 = (l[9]*(phi12_R**2 - phi12_I**2) - 2*l[10]*phi12_R*phi12_I) + (l[11]*(phi13_R**2 - phi13_I**2) - 2*l[12]*phi13_R*phi13_I) + (l[13]*(phi23_R**2 - phi23_I**2) - 2*l[14]*phi23_R*phi23_I)
    g4 = 2*(l[15]*phi12_R - l[16]*phi12_I)*phi1 + 2*(l[17]*phi12_R - l[18]*phi12_I)*phi2 + 2*(l[19]*phi12_R - l[20]*phi12_I)*phi3 
    g5 = 2*(l[21]*phi13_R - l[22]*phi13_I)*phi1 + 2*(l[23]*phi13_R - l[24]*phi13_I)*phi2 + 2*(l[25]*phi13_R - l[26]*phi13_I)*phi3 
    g6 = 2*(l[27]*phi23_R - l[28]*phi23_I)*phi1 + 2*(l[29]*phi23_R - l[30]*phi23_I)*phi2 + 2*(l[31]*phi23_R - l[32]*phi23_I)*phi3 
    g7 = 2*(l[33]*(phi12_R*phi13_R - phi12_I*phi13_I) - l[34]*(phi12_R*phi13_I + phi12_I*phi13_R )) + 2*(l[35]*(phi23_R*phi12_R + phi23_I*phi12_I) - l[36]*(-phi23_R*phi12_I + phi23_I*phi12_R ))
    g8 = 2*(l[37]*(phi13_R*phi23_R - phi13_I*phi23_I) + l[38]*(phi13_R*phi23_I + phi13_I*phi23_R )) + 2*(l[39]*(phi12_R*phi13_R + phi12_I*phi13_I) - l[40]*(-phi12_R*phi13_I + phi12_I*phi13_R ))
    g9 = 2*(l[41]*(phi23_R*phi12_R - phi23_I*phi12_I) - l[42]*(phi23_R*phi12_I + phi23_I*phi12_R )) + 2*(l[43]*(phi13_R*phi23_R + phi13_I*phi23_I) - l[44]*(phi13_R*phi23_I - phi13_I*phi23_R))
    
    
    return g1+g2+g3+g4+g5+g6+g7+g8+g9

In [4]:
@njit(cache = True)
def v2(x,l):
    t,a,b,y,d,e,n = x[0], x[1], x[2], x[3], x[4], x[5], x[6]
    
    phi1 = (s(t)*s(a)*s(b)*s(y))**2
    phi2 = c(t)**2 + (s(t)*c(a))**2
    phi3 = (s(t)*s(a))**2*(c(b)**2 + (s(b)*c(y))**2)
    phi12_R =  1/2*(s(b)*s(y)*(s(2*t)*s(a)*c(d) + s(t)**2*s(2*a)*s(d)*s(e)*c(n)))
    phi12_I = -1/2*(s(b)*s(y)*s(d)*(s(2*t)*s(a)*c(e) + s(t)**2*s(2*a)*s(e)*s(n)))
    phi13_R = 1/2*(s(t)*s(a))**2*s(d)*s(e)*(s(2*b)*s(y)*c(n) + s(b)**2*s(2*y)*s(n))
    phi13_I = 1/2*(s(t)*s(a))**2*s(d)*s(e)*(s(2*b)*s(y)*s(n) - s(b)**2*s(2*y)*c(n))
    phi23_R = 1/2*s(t)**2*s(2*a)*c(b)
    phi23_I = 1/2*s(t)**2*s(2*a)*s(b)*c(y)
    
    
    g1 = 1/2*l[0]*phi1**2 + 1/2*l[1]*phi2**2 + 1/2*l[2]*phi3**2 + l[3]*phi1*phi2 + l[4]*phi1*phi3 + l[5]*phi2*phi3
    g2 = l[6]*(phi12_R**2 + phi12_I**2) + l[7]*(phi13_R**2 + phi13_I**2) + l[8]*(phi23_R**2 + phi23_I**2)
    g3 = (l[9]*(phi12_R**2 - phi12_I**2) - 2*l[10]*phi12_R*phi12_I) + (l[11]*(phi13_R**2 - phi13_I**2) - 2*l[12]*phi13_R*phi13_I) + (l[13]*(phi23_R**2 - phi23_I**2) - 2*l[14]*phi23_R*phi23_I)
    g4 = 2*(l[15]*phi12_R - l[16]*phi12_I)*phi1 + 2*(l[17]*phi12_R - l[18]*phi12_I)*phi2 + 2*(l[19]*phi12_R - l[20]*phi12_I)*phi3 
    g5 = 2*(l[21]*phi13_R - l[22]*phi13_I)*phi1 + 2*(l[23]*phi13_R - l[24]*phi13_I)*phi2 + 2*(l[25]*phi13_R - l[26]*phi13_I)*phi3 
    g6 = 2*(l[27]*phi23_R - l[28]*phi23_I)*phi1 + 2*(l[29]*phi23_R - l[30]*phi23_I)*phi2 + 2*(l[31]*phi23_R - l[32]*phi23_I)*phi3 
    g7 = 2*(l[33]*(phi12_R*phi13_R - phi12_I*phi13_I) - l[34]*(phi12_R*phi13_I + phi12_I*phi13_R )) + 2*(l[35]*(phi23_R*phi12_R + phi23_I*phi12_I) - l[36]*(-phi23_R*phi12_I + phi23_I*phi12_R ))
    g8 = 2*(l[37]*(phi13_R*phi23_R - phi13_I*phi23_I) + l[38]*(phi13_R*phi23_I + phi13_I*phi23_R )) + 2*(l[39]*(phi12_R*phi13_R + phi12_I*phi13_I) - l[40]*(-phi12_R*phi13_I + phi12_I*phi13_R ))
    g9 = 2*(l[41]*(phi23_R*phi12_R - phi23_I*phi12_I) - l[42]*(phi23_R*phi12_I + phi23_I*phi12_R )) + 2*(l[43]*(phi13_R*phi23_R + phi13_I*phi23_I) - l[44]*(phi13_R*phi23_I - phi13_I*phi23_R))
    
    
    return g1+g2+g3+g4+g5+g6+g7+g8+g9

In [5]:
@njit(cache = True)
def v3(x,l):
    t,a,b,y,d,e,n = x[0], x[1], x[2], x[3], x[4], x[5], x[6]
    
    phi1 = (s(t)*s(a)*s(b)*s(y))**2
    phi2 = (s(t)*s(a))**2*(c(b)**2 + (s(b)*c(y))**2) 
    phi3 = c(t)**2 + (s(t)*c(a))**2
    phi12_R = 1/2*(s(t)*s(a))**2*s(d)*s(e)*(s(2*b)*s(y)*c(n) + s(b)**2*s(2*y)*s(n))
    phi12_I = 1/2*(s(t)*s(a))**2*s(d)*s(e)*(s(2*b)*s(y)*s(n) - s(b)**2*s(2*y)*c(n))
    phi13_R = 1/2*(s(b)*s(y)*(s(2*t)*s(a)*c(d) + s(t)**2*s(2*a)*s(d)*s(e)*c(n)))
    phi13_I = -1/2*(s(b)*s(y)*s(d)*(s(2*t)*s(a)*c(e) + s(t)**2*s(2*a)*s(e)*s(n))) 
    phi23_R = 1/2*s(t)**2*s(2*a)*c(b)
    phi23_I = -1/2*s(t)**2*s(2*a)*s(b)*c(y)
    
    
    g1 = 1/2*l[0]*phi1**2 + 1/2*l[1]*phi2**2 + 1/2*l[2]*phi3**2 + l[3]*phi1*phi2 + l[4]*phi1*phi3 + l[5]*phi2*phi3
    g2 = l[6]*(phi12_R**2 + phi12_I**2) + l[7]*(phi13_R**2 + phi13_I**2) + l[8]*(phi23_R**2 + phi23_I**2)
    g3 = (l[9]*(phi12_R**2 - phi12_I**2) - 2*l[10]*phi12_R*phi12_I) + (l[11]*(phi13_R**2 - phi13_I**2) - 2*l[12]*phi13_R*phi13_I) + (l[13]*(phi23_R**2 - phi23_I**2) - 2*l[14]*phi23_R*phi23_I)
    g4 = 2*(l[15]*phi12_R - l[16]*phi12_I)*phi1 + 2*(l[17]*phi12_R - l[18]*phi12_I)*phi2 + 2*(l[19]*phi12_R - l[20]*phi12_I)*phi3 
    g5 = 2*(l[21]*phi13_R - l[22]*phi13_I)*phi1 + 2*(l[23]*phi13_R - l[24]*phi13_I)*phi2 + 2*(l[25]*phi13_R - l[26]*phi13_I)*phi3 
    g6 = 2*(l[27]*phi23_R - l[28]*phi23_I)*phi1 + 2*(l[29]*phi23_R - l[30]*phi23_I)*phi2 + 2*(l[31]*phi23_R - l[32]*phi23_I)*phi3 
    g7 = 2*(l[33]*(phi12_R*phi13_R - phi12_I*phi13_I) - l[34]*(phi12_R*phi13_I + phi12_I*phi13_R )) + 2*(l[35]*(phi23_R*phi12_R + phi23_I*phi12_I) - l[36]*(-phi23_R*phi12_I + phi23_I*phi12_R ))
    g8 = 2*(l[37]*(phi13_R*phi23_R - phi13_I*phi23_I) + l[38]*(phi13_R*phi23_I + phi13_I*phi23_R )) + 2*(l[39]*(phi12_R*phi13_R + phi12_I*phi13_I) - l[40]*(-phi12_R*phi13_I + phi12_I*phi13_R ))
    g9 = 2*(l[41]*(phi23_R*phi12_R - phi23_I*phi12_I) - l[42]*(phi23_R*phi12_I + phi23_I*phi12_R )) + 2*(l[43]*(phi13_R*phi23_R + phi13_I*phi23_I) - l[44]*(phi13_R*phi23_I - phi13_I*phi23_R))
    
    
    return g1+g2+g3+g4+g5+g6+g7+g8+g9

In [6]:
@njit(cache = True)
def v4(x,l):
    t,a,b,y,d,e,n = x[0], x[1], x[2], x[3], x[4], x[5], x[6]
    
    phi1 = c(t)**2 + (s(t)*c(a))**2 
    phi2 = (s(t)*s(a)*s(b)*s(y))**2
    phi3 = (s(t)*s(a))**2*(c(b)**2 + (s(b)*c(y))**2)
    phi12_R = 1/2*(s(b)*s(y)*(s(2*t)*s(a)*c(d) + s(t)**2*s(2*a)*s(d)*s(e)*c(n)))
    phi12_I = 1/2*(s(b)*s(y)*s(d)*(s(2*t)*s(a)*c(e) + s(t)**2*s(2*a)*s(e)*s(n))) 
    phi13_R = 1/2*s(t)**2*s(2*a)*c(b)
    phi13_I = 1/2*s(t)**2*s(2*a)*s(b)*c(y)
    phi23_R = 1/2*(s(t)*s(a))**2*s(d)*s(e)*(s(2*b)*s(y)*c(n) + s(b)**2*s(2*y)*s(n))
    phi23_I = 1/2*(s(t)*s(a))**2*s(d)*s(e)*(s(2*b)*s(y)*s(n) - s(b)**2*s(2*y)*c(n))
    
    
    g1 = 1/2*l[0]*phi1**2 + 1/2*l[1]*phi2**2 + 1/2*l[2]*phi3**2 + l[3]*phi1*phi2 + l[4]*phi1*phi3 + l[5]*phi2*phi3
    g2 = l[6]*(phi12_R**2 + phi12_I**2) + l[7]*(phi13_R**2 + phi13_I**2) + l[8]*(phi23_R**2 + phi23_I**2)
    g3 = (l[9]*(phi12_R**2 - phi12_I**2) - 2*l[10]*phi12_R*phi12_I) + (l[11]*(phi13_R**2 - phi13_I**2) - 2*l[12]*phi13_R*phi13_I) + (l[13]*(phi23_R**2 - phi23_I**2) - 2*l[14]*phi23_R*phi23_I)
    g4 = 2*(l[15]*phi12_R - l[16]*phi12_I)*phi1 + 2*(l[17]*phi12_R - l[18]*phi12_I)*phi2 + 2*(l[19]*phi12_R - l[20]*phi12_I)*phi3 
    g5 = 2*(l[21]*phi13_R - l[22]*phi13_I)*phi1 + 2*(l[23]*phi13_R - l[24]*phi13_I)*phi2 + 2*(l[25]*phi13_R - l[26]*phi13_I)*phi3 
    g6 = 2*(l[27]*phi23_R - l[28]*phi23_I)*phi1 + 2*(l[29]*phi23_R - l[30]*phi23_I)*phi2 + 2*(l[31]*phi23_R - l[32]*phi23_I)*phi3 
    g7 = 2*(l[33]*(phi12_R*phi13_R - phi12_I*phi13_I) - l[34]*(phi12_R*phi13_I + phi12_I*phi13_R )) + 2*(l[35]*(phi23_R*phi12_R + phi23_I*phi12_I) - l[36]*(-phi23_R*phi12_I + phi23_I*phi12_R ))
    g8 = 2*(l[37]*(phi13_R*phi23_R - phi13_I*phi23_I) + l[38]*(phi13_R*phi23_I + phi13_I*phi23_R )) + 2*(l[39]*(phi12_R*phi13_R + phi12_I*phi13_I) - l[40]*(-phi12_R*phi13_I + phi12_I*phi13_R ))
    g9 = 2*(l[41]*(phi23_R*phi12_R - phi23_I*phi12_I) - l[42]*(phi23_R*phi12_I + phi23_I*phi12_R )) + 2*(l[43]*(phi13_R*phi23_R + phi13_I*phi23_I) - l[44]*(phi13_R*phi23_I - phi13_I*phi23_R))
    
    
    return g1+g2+g3+g4+g5+g6+g7+g8+g9

The Stochastic Local Search Algorithm

In [7]:
@njit(cache = True)
def mid_search(f,l,x,y,s_m = 0.5):
    #initailization
    E = np.zeros((2,7))
    best, best_eval, k = x,y,0
    
    #ADD
    x1 = x + np.random.uniform(-0.0005, 0.0005, size = 7)
    x2 = x + np.random.uniform(-0.0005, 0.0005, size = 7)

    delta_f = np.array([f(x1,l)-y,f(x2,l)-y])
    w = delta_f/np.sum(np.abs(delta_f))
    E[0],E[1] = x1-x,x2-x
    e = -E/np.sqrt(np.sum(E*E))
    
    v = w[0]*e[0]+w[1]*e[1]
    
    for i in range(1000):
        if k == 1 or s_m < 0.1:
            break
        
        #Descent
        step = np.random.uniform(0,s_m)
        c = x + step*v
        y_c = f(c,l)
        
        if y_c < 0:
            k = 1
        elif y_c < best_eval:
            best,best_eval = c,y_c
            x,y = c,y_c
        else:
        
            #ADD
            x1 = x + np.random.uniform(-0.0005, 0.0005, size = 7)
            x2 = x + np.random.uniform(-0.0005, 0.0005, size = 7)
    
            delta_f = np.array([f(x1,l)-y,f(x2,l)-y])
            w = delta_f/np.sum(np.abs(delta_f))
            E[0],E[1] = x1-x,x2-x
            e = -E/np.sqrt(np.sum(E*E))
            
            v = w[0]*e[0]+w[1]*e[1]
            
            #Descent
            step = np.random.uniform(0,s_m)
            c = x + step*v
            y_c = f(c,l)
            
            if y_c < 0:
                k = 1
            elif y_c < best_eval:
                best,best_eval = c,y_c
                x,y = c,y_c
            else:
                #update step
                s_m = s_m/1.05
                       
    return (k,best,best_eval)

The Global Optimization Algorithm

In [8]:
def Minimization(f,l, tolerance, ste, T, zmax):
    #initial parameters
    T0 = T
    step = ste
    k = 0
    diff = 100
    l1 = (1,l)
    l1= l1[1:]
    rng = default_rng()
    
    #necessary conditions
    if  l[0] < 0 or l[1] < 0 or l[2] < 0:
        k = 1
    elif l[3] < -math.sqrt(l[0]*l[1]) or l[4] < -math.sqrt(l[0]*l[2]) or l[5] < -math.sqrt(l[1]*l[2]):
        k = 1
      
    # generate an initial point
    best = rng.random(size = 7)*2*pi         
    # evaluate the initial point
    best_eval = f(best,l)
    # current working solution
    curr, curr_eval = best, best_eval
    best_min, best_eval_min = best,best_eval
    
    if best_eval < 0:
        k = 1
        
    else:
        # run the algorithm
        for z in range(zmax):
            if k == 1:
                break
                   
            for i in range(100):
                if k == 1 or abs(diff) < tolerance:
                    break
                              
                # take a step
                candidate = (curr + rng.normal(loc = 0, scale = step, size = 7))%(2*pi)
                # evaluate candidate point
                candidate_eval = f(candidate,l)
                if candidate_eval < 0:
                    k = 1
                #check for new best solution
                else:
                    if candidate_eval < best_eval:
                        best, best_eval = candidate, candidate_eval
                        #calculate temperatute for current epoch
                        if T0 > 0.1:
                            T0 = T0 /(1+((T0**0.5)*math.log(1.5))/(3*step))        
                   
                        
                #difference between candidate and current point evaluation
                diff = candidate_eval - curr_eval
                # calculate metropolis acceptance criterion
                metropolis = exp(-diff / T0)
                # check if we should keep the new point
                if diff < 0 or rng.random() < metropolis:
                    curr, curr_eval = candidate, candidate_eval
                    
                    if i > 20:
                        k,candidate,candidate_eval = mid_search(f,l,curr,curr_eval)
                        
                        if candidate_eval < 0:
                            k = 1
                            
                        elif candidate_eval < curr_eval:
                            curr, curr_eval = candidate, candidate_eval
                            
                        if candidate_eval < best_eval:
                            best, best_eval = candidate, candidate_eval
                            #calculate temperatute for current epoch
                            if T0 > 0.1:
                                T0 = T0 /(1+((T0**0.5)*math.log(1.5))/(3*step))
                                
               
                #calculate the step for the current epoch
                if step > 0.1:
                    step = step/1.02
          
            if k == 0:
                res = minimize(f, best, args=l1, tol = tolerance, method = "COBYLA", 
                options = {'rhobeg': 0.5})
                    
                if res.fun < 0:
                      k = 1
                elif res.fun < best_eval_min:
                     best_min,best_eval_min = res.x, res.fun
                      
            #restart
            diff = 100
            T0 = T
            step = ste
            
            # generate an initial point
            curr = rng.random(size = 7)*2*pi
            #evaluate initial point
            curr_eval = f(curr,l)
            # generate a best point
            best = rng.random(size = 7)*2*pi
            #evaluate best point
            best_eval = f(best,l) 
        
        
            if curr_eval < 0 or best_eval < 0:
                k = 1  
    
    if k == 0:
        res = minimize(f, best_min, args=l1, tol = tolerance, method = "COBYLA", 
        options = {'rhobeg': 0.5})
            
        if res.fun < 0:
              k = 1
                         
    l = np.append(l,k)
                                                                          
    return l

Adjustable parameters

Runs the algorithm

In [ ]:
lista, lista2 = [],[]

data = np.loadtxt(txt, dtype = float)

for i in data:
    lista.append(i)

results = Parallel(n_jobs = -1)(delayed(Minimization)(v,j,tol,3,100,res) for j in lista)
lista.clear()
    
for r in results:
    if r[45] == 0:
        lista.append(r[:45])
    else:
        lista2.append(r)
            
    
results2 = Parallel(n_jobs = -1)(delayed(Minimization)(v2,j,tol,3,100,res) for j in lista)
lista.clear()

for r in results2:
    if r[45] == 0:
        lista.append(r[:45])
    else:
        lista2.append(r)

            
results3 = Parallel(n_jobs = -1)(delayed(Minimization)(v3,j,tol,3,100,res) for j in lista)
lista.clear()

for r in results3:
    if r[45] == 0:
        lista.append(r[:45])
    else:
        lista2.append(r)
            
            
results4 = Parallel(n_jobs = -1)(delayed(Minimization)(v4,j,tol,3,100,res) for j in lista)
lista.clear()

for r in results4:
    if r[45] == 0:
        lista.append(r)
    else:
        lista2.append(r)
        
lista3 = lista1+lista2

np.savetxt(output, np.array(lista3))

lista.clear()   
lista2.clear()
lista3.clear()